In [1]:
import fix_yahoo_finance as yf
import pandas as pd
import requests
import pymongo
import datetime

In [2]:
# Extracted from https://www.nasdaq.com/screening/company-list.aspx
file_path = "Resources/companylist.csv"
df = pd.read_csv(file_path)
df = df[["Symbol","Name","LastSale","MarketCap","IPOyear","Sector","industry","Summary Quote"]]




df = df.loc[df["Sector"]=="Finance"]
df = df.iloc[0:50,:]
start_date = "2004-05-03"
end_date = "2019-05-03"



df.head()

,Symbol,Name,LastSale,MarketCap,IPOyear,Sector,industry,Summary Quote
20,ABM,ABM Industries Incorporated,38.36,$2.54B,NaN,Finance,Diversified Commercial Services,https://www.nasdaq.com/symbol/abm


In [3]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [4]:
# Define database and collection
db = client.EcoParameters_db

db.StockPrice.drop()

collection = db.StockPrice



#df.Date = pd.to_datetime(df.Date)
#df.set_index('Date', inplace=True)
#df.resample('QS').sum()


In [7]:
tickers = list(df["Symbol"])



for ticker in tickers:
    #try:
     #   data = yf.download(ticker, start=start_date, end=end_date)    
     #   flag = True
    #except Exception as e:
    #    print(e)
    #    flag = False
    
    data = yf.download(ticker, start=start_date, end=end_date)
    
    
    try:
        data = data.resample('QS').mean()
        data.reset_index(inplace=True)
        data["Ticker"] = ticker
        data = data[["Ticker","Date","Close","Volume"]]
        list_dict_prices_stock = []
        for item, row in data.iterrows():
            dict_row ={
                "Ticker" : row["Ticker"],
                "Date" : row["Date"].strftime('%Y-%m-%d'),
                "Close" : row["Close"],
                "Volume" : row["Volume"]
                }
            list_dict_prices_stock.append(dict_row)
        db.StockPrice.insert_many(list_dict_prices_stock)
    except Exception as e:
        print(e)
        
    
    

[*********************100%***********************]  1 of 1 downloaded
                 Open       High        Low      Close  Adj Close  \
Date                                                                
2004-04-01  18.601951  18.873171  18.454390  18.706585  13.273659   
2004-07-01  18.565156  18.830469  18.395781  18.652344  13.298750   
2004-10-01  20.673906  20.948906  20.454219  20.714687  14.842500   
2005-01-01  18.679016  18.922131  18.503607  18.685574  13.460000   
2005-04-01  18.915625  19.139219  18.713281  18.937656  13.721094   
2005-07-01  19.870000  20.093281  19.649219  19.902344  14.498594   
2005-10-01  20.266032  20.475714  19.980794  20.249683  14.823651   
2006-01-01  18.740645  18.939677  18.572742  18.761774  13.809516   
2006-04-01  17.522381  17.697778  17.305873  17.511429  12.969683   
2006-07-01  17.484286  17.679524  17.278095  17.502698  13.054127   
2006-10-01  20.587143  20.803810  20.422381  20.654762  15.486667   
2007-01-01  25.710984  26.011475 

In [8]:
list_price = list(db.StockPrice.find({"Ticker":"ABM"}))
print(list_price)

[{'_id': ObjectId('5ccdbdbecb55cf4fd4001ad6'), 'Ticker': 'ABM', 'Date': '2004-04-01', 'Close': 18.706585365853655, 'Volume': 161070.73170731709}, {'_id': ObjectId('5ccdbdbecb55cf4fd4001ad7'), 'Ticker': 'ABM', 'Date': '2004-07-01', 'Close': 18.652343749999996, 'Volume': 113276.5625}, {'_id': ObjectId('5ccdbdbecb55cf4fd4001ad8'), 'Ticker': 'ABM', 'Date': '2004-10-01', 'Close': 20.714687499999997, 'Volume': 138393.75}, {'_id': ObjectId('5ccdbdbecb55cf4fd4001ad9'), 'Ticker': 'ABM', 'Date': '2005-01-01', 'Close': 18.685573770491803, 'Volume': 167681.96721311475}, {'_id': ObjectId('5ccdbdbecb55cf4fd4001ada'), 'Ticker': 'ABM', 'Date': '2005-04-01', 'Close': 18.93765625, 'Volume': 132632.8125}, {'_id': ObjectId('5ccdbdbecb55cf4fd4001adb'), 'Ticker': 'ABM', 'Date': '2005-07-01', 'Close': 19.90234375, 'Volume': 160882.8125}, {'_id': ObjectId('5ccdbdbecb55cf4fd4001adc'), 'Ticker': 'ABM', 'Date': '2005-10-01', 'Close': 20.24968253968254, 'Volume': 154853.96825396825}, {'_id': ObjectId('5ccdbdbecb5